In [1]:
import pandas as pd
import pandasql as psql
# import openpyxl
import os

In [2]:
source_path = r'../source/to_be_extraced.xlsx'
df_rqm = pd.read_excel(source_path,sheet_name='requirement')
df_src = pd.read_excel(source_path,sheet_name='source_table')
df_ext = pd.read_excel(source_path,sheet_name='src_ext_info')

df_rqm

,content_type,extracted_field
0,A,"col_1,col_2,col_3,col_4"
1,B,"col_2,col_3,col_4,col_5"
2,C,"col_3,col_4,col_5,col_6"
3,D,"col_4,col_5,col_6,col_7"
4,E,"col_5,col_6,col_7,col_8"
5,F,"col_6,col_7,col_8,col_9"


In [3]:
df_ext['src_ext_info'] = df_ext.apply(lambda row: row.drop('content_id').to_json(),axis=1 )
df_ext.head()

,content_id,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,src_ext_info
0,content_1,content_1_1,content_1_2,content_1_3,content_1_4,content_1_5,content_1_6,content_1_7,content_1_8,content_1_9,"{""col_1"":""content_1_1"",""col_2"":""content_1_2"",""..."
1,content_2,content_2_1,content_2_2,content_2_3,content_2_4,content_2_5,content_2_6,content_2_7,content_2_8,content_2_9,"{""col_1"":""content_2_1"",""col_2"":""content_2_2"",""..."
2,content_3,content_3_1,content_3_2,content_3_3,content_3_4,content_3_5,content_3_6,content_3_7,content_3_8,content_3_9,"{""col_1"":""content_3_1"",""col_2"":""content_3_2"",""..."
3,content_4,content_4_1,content_4_2,content_4_3,content_4_4,content_4_5,content_4_6,content_4_7,content_4_8,content_4_9,"{""col_1"":""content_4_1"",""col_2"":""content_4_2"",""..."
4,content_5,content_5_1,content_5_2,content_5_3,content_5_4,content_5_5,content_5_6,content_5_7,content_5_8,content_5_9,"{""col_1"":""content_5_1"",""col_2"":""content_5_2"",""..."


In [4]:
df_src_new = pd.merge(df_src,df_ext,how='left',on='content_id')[['content_id','content_type','src_ext_info_y']].rename(columns={'src_ext_info_y':'src_ext_info'})
df_src_new.head()

,content_id,content_type,src_ext_info
0,content_1,A,"{""col_1"":""content_1_1"",""col_2"":""content_1_2"",""..."
1,content_2,B,"{""col_1"":""content_2_1"",""col_2"":""content_2_2"",""..."
2,content_3,C,"{""col_1"":""content_3_1"",""col_2"":""content_3_2"",""..."
3,content_4,D,"{""col_1"":""content_4_1"",""col_2"":""content_4_2"",""..."
4,content_5,E,"{""col_1"":""content_5_1"",""col_2"":""content_5_2"",""..."


In [5]:
psql.sqldf("select *,json_extract(src_ext_info,'$.col_1') from df_src_new where content_id = 'content_1'")

,content_id,content_type,src_ext_info,"json_extract(src_ext_info,'$.col_1')"
0,content_1,A,"{""col_1"":""content_1_1"",""col_2"":""content_1_2"",""...",content_1_1


In [6]:
target_path = r'../target/query_result.xlsx'
if os.path.exists(target_path):
    os.remove(target_path)

with pd.ExcelWriter(target_path,engine='openpyxl') as writer:
    for i, row in df_rqm.iterrows():
        temp_query = ""
        for field in row['extracted_field'].split(','):
            temp_query = temp_query + ", json_extract(src_ext_info,'$." + field + "') AS " + field 
        query = "SELECT content_id, content_type,src_ext_info " + temp_query + " FROM df_src_new WHERE content_type = '" + row['content_type'] + "';"  # where categories is not null
        # print(query) 
        psql.sqldf(query=query).to_excel(writer,sheet_name=row['content_type'],index=False)